In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("spark://spark-master:7077")\
    .appName("prova iniziale")\
    .config("spark.executor.memory", "512m")\
    .getOrCreate()

sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/09/01 17:10:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# sc.setLogLevel("ERROR")

In [3]:
import numpy as np
import pandas as pd
import random
import sklearn.datasets #va installato
import time
import matplotlib.pyplot as plt

from pyspark.sql import Row
from pyspark.sql.functions import col
from pyspark.sql.functions import least
from pyspark.sql.functions import mean
from pyspark.sql.functions import stddev
from pyspark.sql.functions import rand
from pyspark.ml.feature import StandardScaler


In [4]:
%%time

prova =  sklearn.datasets.fetch_kddcup99(percent10 = True, as_frame = True)
X = prova.data

n_cols = 38

CPU times: user 5.86 s, sys: 1.59 s, total: 7.45 s
Wall time: 7.5 s


In [5]:
# Function to retrieve a single row.... copied from the internet
def getrows(df, rownums=None): 
    return df.rdd.zipWithIndex().filter(lambda x: x[1] in rownums).map(lambda x: x[0])

# Define the function for the distance between the cluster centers and the pandas dataframe
def distance(xrow, centers, num_cols=n_cols):
    '''Distance between a dataframe.row
    and the broadcasted value list of centers
    in the form of dataframe.row.value:
    
    broadC = sc.broadcast(center_rows).value
    xrow = clean_X.collect()[1]
    '''
    x = np.array(xrow)[:num_cols]
    the_ds = np.zeros(len(centers))
    
    for c in range(len(centers)):
        c_array = np.array(centers[c])[:num_cols]
        dist2 = np.linalg.norm(x - c_array)**2
        the_ds[c] = dist2
        
    return np.min(the_ds)

def find_closest_center(xrow,broadC):
    '''Distance between a dataframe.row
    and the broadcasted list of centers
    in the form of dataframe.row:
    
    broadC = sc.broadcast(center_rows)
    xrow = clean_X.collect()[1]
    '''
    x = np.array(xrow)
    centers = broadC.value
    the_ds = np.zeros(len(centers))
    
    for c in range(len(centers)):
        c_array = np.array(centers[c])
        dist2 = np.linalg.norm(x - c_array)**2
        the_ds[c] = dist2
        
    return np.argmin(the_ds)

def evaluate_l(log_phi, k, G):
    return G * k/log_phi # G = over-oversampling factor

# Function to select a row based on its probability
def select_row(x):
    if x > np.random.uniform(low = 0, high = 1):
        return True
    else:
        return False
    
def if_greater(xrow, centers):
    return min(distance(xrow, centers, 38), xrow.__getitem__('minimum_cost'))

In [ ]:
%%time

# Hyperparameters
# ks =      [4,   20,    60,    80,     200,    300]
# len_dfs = [100, 1_000, 5_000, 10_000, 50_000, 100_000]

ks =      [4]
len_dfs = [2_023]

# ks =      [4,   4,     4,     4,      4,      4,       4]
# len_dfs = [100, 1_000, 5_000, 10_000, 50_000, 100_000, 494_021]
Gs = list(np.full(len(ks), 3)) # Giosu factor

cycle = list(zip(ks, Gs, len_dfs))
times = []

for step in cycle:
    time_zero = time.time()
    k = step[0]
    G = step[1]
    len_df = step[2]

    X_smaller = X.iloc[random.sample(range(0, len(X.index)), len_df)]
    spark_X = spark.createDataFrame(X_smaller)
    spark_X.persist() # persist in the memory

    n_rows = spark_X.count()
    n_cols = len(getrows(spark_X, rownums=[0]).collect()[0])

    col_type = np.array(spark_X.dtypes)
    types = col_type[:,1] 
    colnames = col_type[:,0]
    clean_X = spark_X.select([col(colnames[i]) for i in range(len(colnames)) if not types[i] == 'binary'])

    colnames = list(clean_X.dtypes[i][0] for i in range(len(clean_X.dtypes)))

    random_n = [np.random.randint(0, n_rows)]
    random_sample = getrows(clean_X, random_n).collect()

    clean_X = (clean_X.select('*')
               .withColumn('minimum_cost', sum((col(colname)-random_sample[0][colname])**2 for colname in colnames)))

    initial_cost = np.log(clean_X.agg({"minimum_cost": "sum"}).collect()[0][0])
    n_iter = int(initial_cost)

    temp = random_sample[0].asDict()
    temp["minimum_cost"] = 0
    random_sample[0] = Row(**temp)

    bCent = sc.broadcast(random_sample)

    phi_iter = initial_cost
    l = evaluate_l(phi_iter, k, G)

    i = 0
    last_centers = 1
    start_time = time.time()

    # To be removed
    new_centers = []
    time_new_rows = []
    time_broadc = []
    time_update = []

    while i < n_iter:# or len(bCent.value) < k:

        '''
        Nel ciclo for:
            - Evaluate for each row l * d()^2 / phi
            - Sample with that probability
            - Broadcast centers to nodes
            - Evaluate new cost

        Ricordiamoci che distance è già al quadrato e che phi è il logaritmo
        '''

        # Evaluate the probability and select the new rows
        new_rows = clean_X.rdd\
                   .filter(lambda row: select_row(np.exp(np.log(row.__getitem__('minimum_cost'))-phi_iter)*l)).collect()
        # To be removed
        time_new_rows.append((time.time() - start_time))
        start_time = time.time()

#         # Evaluate the probability and select the new rows
#         new_rows = clean_X.select('*').withColumn('random_number', rand(seed=4245))\
#                           .filter(col('random_number') < col('minimum_cost')).drop('random_number').collect()
#         # To be removed
#         time_new_rows.append((time.time() - start_time))
#         start_time = time.time()
        
        # Update the broadcast
        bCent = sc.broadcast(bCent.value + new_rows)
        # To be removed
        time_broadc.append((time.time() - start_time))
        start_time = time.time()

        # Update the minimum distance
        if len(new_rows) == 1:
            clean_X = clean_X.select('*').withColumn('minimum_cost', least('minimum_cost', sum((col(colname)-bCent.value[0][colname])**2 for colname in colnames) ))

        elif len(new_rows) > 1:
            clean_X = clean_X.select('*').withColumn('dummy', least(*[sum((col(colname)-bCent.value[center][colname])**2 for colname in colnames) for center in range(len(new_rows))] )).\
                      withColumn('minimum_cost', least('minimum_cost', 'dummy')).\
                      drop('dummy')

        last_centers = len(bCent.value)

        # Evaluate new cost
        phi_iter = np.log(clean_X.agg({"minimum_cost": "sum"}).collect()[0][0])

        # To be removed
        time_update.append((time.time() - start_time))
        start_time = time.time()

        i += 1
        # To be removed
        new_centers.append(len(new_rows))

    fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(12, 4))
    ax1.plot(new_centers, 'o--', label='# new centers')
    ax1.set_xlabel('# iteration')
    ax1.grid()

    ax2.plot(time_new_rows, 'o--', label='new rows')
    ax2.plot(time_broadc, 'o--', label='broadcast')
    ax2.plot(time_update, 'o--', label='update')
    ax2.set_xlabel('# iteration')
    ax2.legend()
    ax2.grid()

    figname = 'figures/fig_k' + str(step[0]) + '_G' + str(step[1]) + '_len' + str(step[2])
    plt.savefig(figname)
    
    times.append(time.time() - time_zero)
    print('\nIn configuration (k, G, len_df) =', step, ' time spent is', times[-1], 'seconds')
    print('Found', len(bCent.value), 'centers in', n_iter, 'iterations')
    print('Dataframe had', n_rows, 'rows and', n_cols, 'columns')
    
    bCent.destroy()
    spark_X.unpersist() # to unpersist the memory in the workers



In [24]:
%%time

X_smaller = X.iloc[random.sample(range(0, len(X.index)), 494_021)]
spark_X = spark.createDataFrame(X_smaller)

col_type = np.array(spark_X.dtypes)
types = col_type[:,1] 
colnames = col_type[:,0]
clean_X = spark_X.select([col(colnames[i]) for i in range(len(colnames)) if not types[i] == 'binary'])
colnames = list(clean_X.dtypes[i][0] for i in range(len(clean_X.dtypes)))

clean_X = (clean_X.select('*')
           .withColumn('minimum_cost', sum((col(colname))**2 for colname in colnames)))


CPU times: user 2min 47s, sys: 419 ms, total: 2min 47s
Wall time: 2min 48s


In [33]:
initial_cost = np.log(clean_X.agg({"minimum_cost": "sum"}).collect()[0][0])

23/09/01 17:26:20 WARN TaskSetManager: Stage 19 contains a task of very large size (33178 KiB). The maximum recommended task size is 1000 KiB.


In [34]:
%%time

# spark.driver.maxResultSize
# spark.executor.memory
new_rows = clean_X.rdd.filter(lambda row: select_row(0.5*row['minimum_cost']/np.exp(initial_cost))).collect()


23/09/01 17:26:45 WARN TaskSetManager: Stage 22 contains a task of very large size (33178 KiB). The maximum recommended task size is 1000 KiB.


CPU times: user 14.3 ms, sys: 9.91 ms, total: 24.2 ms
Wall time: 6.46 s


In [35]:
%%time

new_rows = clean_X.rdd.filter(lambda row: select_row(0.5*row.__getitem__('minimum_cost')/np.exp(initial_cost))).collect()


23/09/01 17:26:52 WARN TaskSetManager: Stage 23 contains a task of very large size (33178 KiB). The maximum recommended task size is 1000 KiB.


CPU times: user 31.2 ms, sys: 192 µs, total: 31.3 ms
Wall time: 7.16 s


In [36]:
%%time

new_rows = clean_X.select('*').withColumn('random_number', rand(seed=4245))\
                  .filter(col('random_number') < 0.5*col('minimum_cost')/np.exp(initial_cost)).drop('random_number').collect()


23/09/01 17:26:59 WARN TaskSetManager: Stage 24 contains a task of very large size (33178 KiB). The maximum recommended task size is 1000 KiB.


CPU times: user 33 ms, sys: 224 µs, total: 33.2 ms
Wall time: 4.01 s


In [37]:
new_rows

[Row(duration=2, src_bytes=693375640, dst_bytes=0, land=0, wrong_fragment=0, urgent=0, hot=1, num_failed_logins=0, logged_in=0, num_compromised=0, root_shell=0, su_attempted=0, num_root=0, num_file_creations=0, num_shells=0, num_access_files=0, num_outbound_cmds=0, is_host_login=0, is_guest_login=0, count=57, srv_count=3, serror_rate=0.79, srv_serror_rate=0.67, rerror_rate=0.21, srv_rerror_rate=0.33, same_srv_rate=0.05, diff_srv_rate=0.39, srv_diff_host_rate=0.0, dst_host_count=255, dst_host_srv_count=3, dst_host_same_srv_rate=0.01, dst_host_diff_srv_rate=0.09, dst_host_same_src_port_rate=0.22, dst_host_srv_diff_host_rate=0.0, dst_host_serror_rate=0.18, dst_host_srv_serror_rate=0.67, dst_host_rerror_rate=0.05, dst_host_srv_rerror_rate=0.33, minimum_cost=4.807697781454779e+17)]